In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jan 16 22:26:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
#Check if GPU is in use.
print(f"tensorflow GPU version: {tf.__version__}")

tensorflow GPU version: 2.9.2


In [ ]:
from PIL import Image
from tqdm import tqdm
import os 
import time
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model

In [ ]:
#Clear the GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
!pip install patool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 2.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/CLWD.rar /content/

In [ ]:
import patoolib
patoolib.extract_archive("/content/CLWD.rar", outdir="/content")

patool: Extracting /content/CLWD.rar ...
patool: running /usr/bin/unrar x -- /content/CLWD.rar
patool:     with cwd='/content'
patool: ... /content/CLWD.rar extracted to `/content'.


'/content'

In [ ]:
!cp -r /content/CLWD/train/Watermarked_image /content/Data/Train/WM_Images
!cp -r /content/CLWD/train/Watermark_free_image /content/Data/Train/Original_Ims



In [ ]:
!cp -r /content/CLWD/test/Watermarked_image /content/Data/Test/WM_Images
!cp -r /content/CLWD/test/Watermark_free_image /content/Data/Test/Original_Ims

In [ ]:
def train_augment_generator(train_image_gen, train_wm_gen, dir_path, shape, seed = 1, batch_size = 5):

    train_image_generator = train_image_gen.flow_from_directory(f'{dir_path}Original_Ims/', batch_size = batch_size, seed = seed, target_size=(shape, shape), shuffle=False) 
    train_wm_generator = train_wm_gen.flow_from_directory(f'{dir_path}WM_Images/', batch_size = batch_size, seed = seed, target_size=(shape, shape), shuffle=False)

    while True:
        orig_image = train_image_generator.next() # Original image
        wm_image = train_wm_generator.next() # Marked image 
        yield tf.cast(wm_image[0],tf.float32), tf.cast(orig_image[0], tf.float32) # yeild X = marked image, Y = clean image

In [ ]:
# Configuration
GENERATE_SQUARE = 224 * 2
BATCH_SIZE = 8
SEED = 54
train_path = "/content/Data/Train/"
test_path = "/content/Data/Test/"


# Creating data generators for train and validation images
train_orig_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) 
train_wm_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_augment_generator(train_orig_data_generator, train_wm_data_generator, train_path, GENERATE_SQUARE, batch_size=BATCH_SIZE) 
valid_generator = train_augment_generator(train_orig_data_generator, train_wm_data_generator, test_path, GENERATE_SQUARE, batch_size=BATCH_SIZE)

train_gen = train_orig_data_generator.flow_from_directory(
    directory="/content/Data/Train/WM_Images/",
    target_size=(GENERATE_SQUARE, GENERATE_SQUARE),
    batch_size=BATCH_SIZE,
    seed=SEED)

test_gen = train_orig_data_generator.flow_from_directory(
    directory="/content/Data/Test/WM_Images/",
    target_size=(GENERATE_SQUARE, GENERATE_SQUARE),
    batch_size=BATCH_SIZE)



Found 60000 images belonging to 1 classes.
Found 10000 images belonging to 1 classes.


In [ ]:
#Number of steps per epoch
STEP_SIZE_TRAIN=train_gen.n//BATCH_SIZE
STEP_SIZE_VALID=test_gen.n//BATCH_SIZE


print(STEP_SIZE_TRAIN,STEP_SIZE_VALID)
print(BATCH_SIZE)

7500 1250
8


In [ ]:
class wm_remover(Model):
  def __init__(self, in_shp):
    super(wm_remover, self).__init__()
    # Define the encoder that will perform feature extraction
    self.encoder = tf.keras.Sequential([          
      layers.Input(shape=(in_shp, in_shp, 3)),
      layers.Conv2D(256, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2D(128, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2D(32, (3,3), activation='relu', padding='same', strides=2), 
      ])

    # Define the decoder that will perfoorm image decoding
    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(128, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(256, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(3, kernel_size=(3,3), activation='sigmoid', padding='same')
      ])
    
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true))) # For the evaluation metric, we use mean squared error applied to each pixel of the image

In [ ]:
autoencoder = wm_remover(GENERATE_SQUARE)
autoencoder.compile(optimizer='adam', loss=root_mean_squared_error) 
autoencoder.build((1,GENERATE_SQUARE,GENERATE_SQUARE,3))

In [ ]:
weight_path = "/content/Weights/_20221225-205242__pretrained_weights0015.h5" 
autoencoder.load_weights(weight_path) 

os.makedirs("/content/new_weights/", exist_ok=True)
mc = tf.keras.callbacks.ModelCheckpoint("/content/new_weights/_pretrained_weights{epoch:04d}.h5", save_weights_only=True, save_freq=1)

In [ ]:
# Train the network
autoencoder.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                    callbacks=[mc])